## Configuration


In [0]:
dbutils.widgets.text("catalog", "dev")
dbutils.widgets.text("silver_schema", "02_silver")
dbutils.widgets.text("gold_schema", "03_gold_supply")

In [0]:
catalog = dbutils.widgets.get("catalog")
silver_schema = dbutils.widgets.get("silver_schema")
gold_schema = dbutils.widgets.get("gold_schema")

In [0]:
# catalog = "dev"
# silver_schema = "02_silver"
# gold_schema = "03_gold_supply"

## Create Gold Schema


In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{gold_schema}")

## B2C KPI View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2c_kpi AS
WITH base_data AS (
    SELECT
        i.product_id,
        i.quantity_on_hand,
        p.retail_price,
        coi.total_price,
        coi.quantity,
        coi.order_status,
        coi.order_date,
        coi.order_id AS co_order_id
    FROM {catalog}.{silver_schema}.inventory_mv i
    JOIN {catalog}.{silver_schema}.product_mv p 
        ON i.product_id = p.product_id
    JOIN {catalog}.{silver_schema}.consumer_orders_mv coi 
        ON p.product_id = coi.product_id
),
agg_level1 AS (
    SELECT
        CURRENT_DATE() AS kpi_date,
        SUM(quantity_on_hand * retail_price) AS inventory_value,
        SUM(total_price) AS total_sales_value,
        SUM(quantity) AS total_quantity_sold,
        AVG(quantity_on_hand) AS avg_quantity_on_hand,
        COUNT(DISTINCT co_order_id) AS total_orders,
        SUM(CASE WHEN order_status = 'Returned' THEN 1 ELSE 0 END) AS backorder_count,
        COUNT(*) AS total_order_lines,
        AVG(DATEDIFF(CURRENT_DATE(), order_date)) AS avg_days_sales_inventory,
        DATEDIFF(MAX(order_date), MIN(order_date)) / 7.0 AS total_weeks_in_period
    FROM base_data
)
SELECT
    kpi_date,
    ROUND(inventory_value / NULLIF(total_sales_value, 0), 2) AS stock_to_sales_ratio,
    ROUND(total_quantity_sold / NULLIF(avg_quantity_on_hand, 0), 2) AS inventory_turnover_rate,
    ROUND(avg_quantity_on_hand / NULLIF(total_quantity_sold / NULLIF(total_weeks_in_period, 0), 0), 2) AS weeks_on_hand,
    ROUND(backorder_count * 100.0 / NULLIF(total_order_lines, 0), 2) AS backorder_rate_pct,
    ROUND(avg_days_sales_inventory, 2) AS days_sales_in_inventory
FROM agg_level1;
""")


## B2B KPI View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2b_kpi AS
WITH base_data AS (
    SELECT
        i.product_id,
        i.quantity_on_hand,
        p.retail_price,
        di.invoice_item_total_amount AS total_price,
        di.invoiced_quantity AS quantity,
        di.invoice_date AS order_date,
        di.invoice_id AS co_order_id
    FROM {catalog}.{silver_schema}.inventory_mv i
    JOIN {catalog}.{silver_schema}.product_mv p 
        ON i.product_id = p.product_id
    JOIN {catalog}.{silver_schema}.distributor_sale_order_mv di 
        ON p.product_id = di.product_id
),
agg_level1 AS (
    SELECT
        CURRENT_DATE() AS kpi_date,
        SUM(quantity_on_hand * retail_price) AS inventory_value,
        SUM(total_price) AS total_sales_value,
        SUM(quantity) AS total_quantity_sold,
        AVG(quantity_on_hand) AS avg_quantity_on_hand,
        COUNT(DISTINCT co_order_id) AS total_orders,
        COUNT(*) AS total_order_lines,
        AVG(DATEDIFF(CURRENT_DATE(), order_date)) AS avg_days_sales_inventory,
        DATEDIFF(MAX(order_date), MIN(order_date)) AS total_days
    FROM base_data
)
SELECT
    kpi_date,
    ROUND(inventory_value / NULLIF(total_sales_value, 0), 2) AS stock_to_sales_ratio,
    ROUND(total_quantity_sold / NULLIF(avg_quantity_on_hand, 0), 2) AS inventory_turnover_rate,
    ROUND(avg_quantity_on_hand / NULLIF(total_quantity_sold / NULLIF((total_days / 7.0), 0), 0), 2) AS weeks_on_hand,
    ROUND(avg_days_sales_inventory, 2) AS days_sales_in_inventory
FROM agg_level1;
""")


## Top Inventory Products View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_top_inventory_products AS
SELECT 
    pr.product_name,
    inv.quantity_on_hand
FROM {catalog}.{silver_schema}.inventory_mv inv
JOIN {catalog}.{silver_schema}.product_mv pr
    ON inv.product_id = pr.product_id
ORDER BY inv.quantity_on_hand DESC
LIMIT 10;
""")


## Low Stock Products View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_low_stock_products AS
SELECT p.product_name, inv.quantity_on_hand, inv.reorder_level
FROM {catalog}.{silver_schema}.inventory_mv inv
JOIN {catalog}.{silver_schema}.product_mv p ON inv.product_id = p.product_id
WHERE inv.quantity_on_hand <= inv.reorder_level
ORDER BY inv.quantity_on_hand ASC;
""")


## Inventory Status Summary View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_inventory_status_summary AS
SELECT 
    inventory_status,
    COUNT(*) AS count
FROM {catalog}.{silver_schema}.inventory_mv
GROUP BY inventory_status;
""")


## Location Inventory Value View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_location_inventory_value AS
SELECT 
    inv.location_name,
    inv.state,
    inv.country,
    SUM(inv.quantity_on_hand * pr.retail_price) AS total_stock_value
FROM {catalog}.{silver_schema}.inventory_mv inv
JOIN {catalog}.{silver_schema}.product_mv pr
    ON inv.product_id = pr.product_id
GROUP BY inv.country, inv.state, inv.location_name
ORDER BY total_stock_value DESC;
""")


## Location Inventory Quantity View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_location_inventory_quantity AS
SELECT 
    location_name,
    state,
    country,
    SUM(quantity_on_hand) AS total_quantity
FROM {catalog}.{silver_schema}.inventory_mv
GROUP BY country, state, location_name
ORDER BY total_quantity DESC;
""")


## B2C Monthly Purchase Summary View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2c_monthly_purchase_summary AS
SELECT
    DATE_TRUNC('month', order_date) AS order_month,
    ROUND(SUM(total_price), 2) AS total_purchase_amount
FROM {catalog}.{silver_schema}.consumer_orders_mv
GROUP BY DATE_TRUNC('month', order_date)
ORDER BY order_month;
""")


## B2C Purchase Summary by Location View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2c_purchase_summary_by_location AS
SELECT 
    c.state,
    c.country,
    ROUND(SUM(co.order_total_amount), 2) AS total_purchase_value
FROM {catalog}.{silver_schema}.consumer_mv c
JOIN {catalog}.{silver_schema}.consumer_orders_mv co
    ON c.consumer_id = co.consumer_id
GROUP BY c.state, c.country
ORDER BY total_purchase_value DESC;
""")


## B2C Category Quantity Ordered View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2c_category_quantity_ordered AS
SELECT 
    pr.category,
    SUM(coi.quantity) AS total_quantity_ordered
FROM {catalog}.{silver_schema}.consumer_orders_mv coi
JOIN {catalog}.{silver_schema}.product_mv pr
    ON coi.product_id = pr.product_id
GROUP BY pr.category
ORDER BY total_quantity_ordered DESC;
""")


## B2C Order Status Summary View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2c_order_status_summary AS
SELECT 
    order_status,
    COUNT(*) AS count
FROM {catalog}.{silver_schema}.consumer_orders_mv
GROUP BY order_status;
""")


## B2B Monthly Purchase Summary View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2b_monthly_purchase_summary AS
SELECT
    DATE_TRUNC('month', invoice_date) AS order_month,
    ROUND(SUM(invoice_item_total_amount), 2) AS total_purchase_amount
FROM {catalog}.{silver_schema}.distributor_sale_order_mv
GROUP BY DATE_TRUNC('month', invoice_date)
ORDER BY order_month;
""")


## B2B Top Distributor Purchase Value View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2b_top_distributor_purchase_value AS
SELECT 
    d.distributor_name,
    ROUND(SUM(di.invoice_item_total_amount), 2) AS total_purchase_value
FROM {catalog}.{silver_schema}.distributor_mv d
JOIN {catalog}.{silver_schema}.distributor_sale_order_mv dso
    ON d.distributor_id = dso.distributor_id
JOIN {catalog}.{silver_schema}.distributor_sale_order_mv di
    ON dso.order_id = di.sales_order_id
GROUP BY d.distributor_name
ORDER BY total_purchase_value DESC;
""")


## B2B Category Quantity Ordered View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2b_category_quantity_ordered AS
SELECT 
    pr.category,
    SUM(di.invoiced_quantity) AS total_quantity_ordered
FROM {catalog}.{silver_schema}.distributor_sale_order_mv di
JOIN {catalog}.{silver_schema}.product_mv pr
    ON di.product_id = pr.product_id
GROUP BY pr.category
ORDER BY total_quantity_ordered DESC;
""")


## B2B Order Status Summary View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_b2b_order_status_summary AS
SELECT 
    'Active' AS order_status,
    COUNT(*) AS count
FROM {catalog}.{silver_schema}.distributor_sale_order_mv;
""")


## Overall KPI View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_overall_kpi AS
WITH b2c_base AS (
    SELECT
        i.product_id,
        i.quantity_on_hand,
        p.retail_price,
        coi.total_price,
        coi.quantity AS quantity,
        coi.order_status,
        coi.order_date,
        coi.order_id AS co_order_id
    FROM {catalog}.{silver_schema}.inventory_mv i
    JOIN {catalog}.{silver_schema}.product_mv p 
        ON i.product_id = p.product_id
    JOIN {catalog}.{silver_schema}.consumer_orders_mv coi 
        ON p.product_id = coi.product_id
),
b2b_base AS (
    SELECT
        i.product_id,
        i.quantity_on_hand,
        p.retail_price,
        di.invoice_item_total_amount AS total_price,
        di.invoiced_quantity AS quantity,
        di.invoice_date AS order_date,
        di.invoice_id AS co_order_id,
        NULL AS order_status
    FROM {catalog}.{silver_schema}.inventory_mv i
    JOIN {catalog}.{silver_schema}.product_mv p 
        ON i.product_id = p.product_id
    JOIN {catalog}.{silver_schema}.distributor_sale_order_mv di 
        ON p.product_id = di.product_id
),
combined_base AS (
    SELECT * FROM b2c_base
    UNION ALL
    SELECT * FROM b2b_base
),
agg_level1 AS (
    SELECT
        CURRENT_DATE() AS kpi_date,
        SUM(quantity_on_hand * retail_price) AS inventory_value,
        SUM(total_price) AS total_sales_value,
        SUM(quantity) AS total_quantity_sold,
        AVG(quantity_on_hand) AS avg_quantity_on_hand,
        COUNT(DISTINCT co_order_id) AS total_orders,
        SUM(CASE WHEN order_status = 'Returned' THEN 1 ELSE 0 END) AS backorder_count,
        COUNT(*) AS total_order_lines,
        AVG(DATEDIFF(CURRENT_DATE(), order_date)) AS avg_days_sales_inventory,
        DATEDIFF(MAX(order_date), MIN(order_date)) / 7.0 AS total_weeks_in_period
    FROM combined_base
)
SELECT
    kpi_date,
    ROUND(inventory_value / NULLIF(total_sales_value, 0), 2) AS stock_to_sales_ratio,
    ROUND(total_quantity_sold / NULLIF(avg_quantity_on_hand, 0), 2) AS inventory_turnover_rate,
    ROUND(avg_quantity_on_hand / NULLIF(total_quantity_sold / NULLIF(total_weeks_in_period, 0), 0), 2) AS weeks_on_hand,
    ROUND(backorder_count * 100.0 / NULLIF(total_order_lines, 0), 2) AS backorder_rate_pct,
    ROUND(avg_days_sales_inventory, 2) AS days_sales_in_inventory
FROM agg_level1;
""")


## Overall Monthly Purchase Summary View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_overall_monthly_purchase_summary AS
SELECT 
    order_month,
    ROUND(SUM(total_purchase_amount), 2) AS total_purchase_amount
FROM (
    SELECT order_month, total_purchase_amount
    FROM {catalog}.{gold_schema}.supply_b2c_monthly_purchase_summary
    UNION ALL
    SELECT order_month, total_purchase_amount
    FROM {catalog}.{gold_schema}.supply_b2b_monthly_purchase_summary
) AS combined
GROUP BY order_month
ORDER BY order_month;
""")


## Overall Category Quantity Ordered View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_overall_category_quantity_ordered AS
SELECT 
    category,
    SUM(total_quantity_ordered) AS total_quantity_ordered
FROM (
    SELECT category, total_quantity_ordered
    FROM {catalog}.{gold_schema}.supply_b2c_category_quantity_ordered
    UNION ALL
    SELECT category, total_quantity_ordered
    FROM {catalog}.{gold_schema}.supply_b2b_category_quantity_ordered
) AS combined
GROUP BY category
ORDER BY total_quantity_ordered DESC;
""")


## Overall Order Status Summary View


In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.supply_overall_order_status_summary AS
SELECT 
    order_status,
    SUM(count) AS count
FROM (
    SELECT order_status, count
    FROM {catalog}.{gold_schema}.supply_b2c_order_status_summary
    UNION ALL
    SELECT order_status, count
    FROM {catalog}.{gold_schema}.supply_b2b_order_status_summary
) AS combined
GROUP BY order_status;
""")
